# Elaboracion del modelo para el proyecto SENDA

In [68]:
import pandas as pd
from datetime import datetime
import pandas as pd
import numpy as np
import random
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA

In [69]:
seremi_data = {
    "2014": pd.read_csv("raw_data/seremi_2014.csv"),
    "2015": pd.read_csv("raw_data/seremi_2015.csv"),
    "2016": pd.read_csv("raw_data/seremi_2016.csv"),
    "2017": pd.read_csv("raw_data/seremi_2017.csv"),
    "2018": pd.read_csv("raw_data/seremi_2018.csv"),
    "2019": pd.read_csv("raw_data/seremi_2019.csv"),
    "2020": pd.read_csv("raw_data/seremi_2020.csv"),
    "2021": pd.read_csv("raw_data/seremi_2021.csv"),
    "2022": pd.read_csv("raw_data/seremi_2022.csv"),
    "2023": pd.read_csv("raw_data/seremi_2023.csv"),
    "2024": pd.read_csv("raw_data/seremi_2024.csv"),
    "2025": pd.read_csv("raw_data/seremi_2025.csv"),
}

def procesar_datasets_climaticos(seremi_data_dict):
    def grados_a_cardinal(grados):
        if pd.isna(grados):
            return None
        direcciones = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
        index = int((grados + 22.5) % 360 // 45)
        return direcciones[index]

    for año, df in seremi_data_dict.items():

        if "winddirection_10m_dominant" in df.columns:
            df["wind_direction_cardinal"] = df["winddirection_10m_dominant"].apply(grados_a_cardinal)

        df = df.rename(columns={
            "temperature_2m_max": "Temperatura Máxima",
            "temperature_2m_min": "Temperatura Mínima",
            "precipitation_sum": "Precipitaciones (suma)",
            "winddirection_10m_dominant": "Dirección Viento"
        })

        if "Dirección Viento" in df.columns:
            df = df.drop(columns=["Dirección Viento"])
        if "wind_direction_cardinal" in df.columns:
            df = df.rename(columns={"wind_direction_cardinal": "Dirección del Viento"})

        seremi_data_dict[año] = df

    return seremi_data_dict

seremi_data = procesar_datasets_climaticos(seremi_data)


In [70]:
archivos_filtrados = {
    "2014": "raw_data/2014_filtrado.csv",
    "2015": "raw_data/2015_filtrado.csv",
    "2016": "raw_data/2016_filtrado.csv",
    "2017": "raw_data/2017_filtrado.csv",
    "2018": "raw_data/2018_filtrado.csv",
    "2019": "raw_data/2019_filtrado.csv",
    "2020": "raw_data/2020_filtrado.csv",
    "2021": "raw_data/2021_filtrado.csv",
    "2022": "raw_data/2022_filtrado.csv",
    "2023": "raw_data/2023_filtrado.csv",
    "2024": "raw_data/2024_filtrado.csv",
    "2025": "raw_data/2025_filtrado.csv"
}

filtrado_limpio = {}

meses_abreviados = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']

columnas_a_eliminar = ["Cód. SS/SEREMI", "Cód. Estab.", "Cód. Nivel Cuidado", "Acum", "Año"]

for año, archivo in archivos_filtrados.items():
    df = pd.read_csv(archivo)

    df = df[~df["Nombre Nivel Cuidado"].isin([
      "330 - Area Pensionado",
      "330 - Área Pensionado",
      "401 - Área Médica Adulto Cuidados Básicos",
      "402 - Área Médica Adulto Cuidados Medios",
      "403 - Área Médico-Quirúrgico Cuidados Básicos",
      "404 - Área Médico-Quirúrgico Cuidados Medios",
      "405 - Área Cuidados Intensivos Adultos",
      "406 - Área Cuidados Intermedios Adultos",
      "407 - Área Médica Pediátrica Cuidados Básicos",
      "408 - Área Médica Pediátrica Cuidados Medios",
      "409 - Área Médico-Quirúrgico Pediátrica Cuidados Básicos",
      "410 - Área Médico-Quirúrgico Pediátrica Cuidados Medios",
      "411 - Área Cuidados Intensivos Pediátricos",
      "412 - Área Cuidados Intermedios Pediátricos",
      "413 - Área Neonatología Cuidados Básicos",
      "414 - Área Neonatología Cuidados Intensivos",
      "415 - Área Neonatología Cuidados Intermedios",
      "416 - Área Obstetricia",
      "418 - Área Psiquiatría Adulto Corta estadía",
      "419 - Área Psiquiatría Adulto Mediana estadía",
      "420 - Área Psiquiatría Adulto Larga estadía",
      "421 - Área Psiquiatría Infanto-adolescente corta estadía",
      "427 - Área Sociosanitaria Adulto",
      "428 - Área de Hospitalización de Cuidados Intensivos en Psiquiatría Adulto",
      "429 - Área de Hospitalización de Cuidados Intensivos en Psiquiatría Infanto Adolescente",
      "Área Cuidados Intensivos Adultos",
      "Área Cuidados Intensivos Pediátricos",
      "Área Cuidados Intermedios Adultos",
      "Área Cuidados Intermedios Pediátricos",
      "Área Médica Adulto Cuidados Básicos",
      "Área Médica Adulto Cuidados Medios",
      "Área Médica Pediátrica Cuidados Básicos",
      "Área Médica Pediátrica Cuidados Medios",
      "Área Médico-Quirúrgico Cuidados Básicos",
      "Área Médico-Quirúrgico Cuidados Medios",
      "Área Médico-Quirúrgico Pediátrica Cuidados Básicos",
      "Área Médico-Quirúrgico Pediátrica Cuidados Medios",
      "Área Neonatología Cuidados Básicos",
      "Área Neonatología Cuidados Intensivos",
      "Área Neonatología Cuidados Intermedios",
      "Área Obstetricia",
      "Área Pensionado",
      "Área Psiquiatría Adulto Corta estadía",
      "Área Psiquiatría Adulto Larga estadía",
      "Área Psiquiatría Adulto Mediana estadía",
      "Área Psiquiatría Infanto-adolescente corta estadía"
    ]
    )].copy()

    df = df.drop(columns=columnas_a_eliminar, errors='ignore')

    df = df.rename(columns={
        "Nombre SS/SEREMI": "SEREMI",
        "Nombre Nivel Cuidado": "Area"
    })

    nuevo_nombre_columnas = {}
    año_int = int(año)

    for i, mes_abrev in enumerate(meses_abreviados, start=1):
        if mes_abrev in df.columns:
            fecha = datetime(año_int, i, 1).strftime('%Y-%m-%d')
            nuevo_nombre_columnas[mes_abrev] = fecha

    df = df.rename(columns=nuevo_nombre_columnas)

    filtrado_limpio[año] = df


In [71]:
def extraer_info_hospital_area_todo_el_año(hospital_nombre):
    registros = []

    for año, df in filtrado_limpio.items():
        df_seremi = seremi_data[año].copy()
        df_seremi['Mes'] = pd.to_datetime(df_seremi['Mes'], errors='coerce')
        df_seremi.dropna(subset=['Mes'], inplace=True)
        df_seremi['Mes'] = df_seremi['Mes'].dt.to_period("M").dt.to_timestamp()

        columnas_mes = [col for col in df.columns if col.startswith("20")]
        for col in columnas_mes:
            df_tmp = df[
                (df["Nombre Establecimiento"] == hospital_nombre) &
                (df["Area"] == "Datos Establecimiento")
            ][["SEREMI", "Nombre Establecimiento", "Area", "Glosa", col]].copy()

            if df_tmp.empty:
                continue

            df_tmp = df_tmp.rename(columns={col: "Valor"})
            df_tmp["Fecha"] = pd.to_datetime(col)

            clima_mes = df_seremi[df_seremi["Mes"] == df_tmp["Fecha"].iloc[0]].copy()
            df_merge = df_tmp.merge(clima_mes, on=["SEREMI"], how="left")

            registros.append(df_merge)

    if registros:
        return pd.concat(registros, ignore_index=True).sort_values("Fecha")
    else:
        print("⚠️ No se encontraron datos para ese hospital y área.")
        return pd.DataFrame()

In [72]:
def preparar_dataset_climatico_avanzado(df):
    if 'Area' in df.columns:
        df = df[df['Area'] == "Datos Establecimiento"].copy()

    df_pivot = df.pivot(index='Fecha', columns='Glosa', values='Valor').reset_index()

    columnas_necesarias = ['Dias Cama Disponibles', 'Dias Cama Ocupados',
                           'Promedio Cama Disponibles', 'Numero de Egresos']
    if not all(col in df_pivot.columns for col in columnas_necesarias):
        print("❌ Datos insuficientes para generar el modelo.")
        return None

    clima = df.drop_duplicates(subset='Fecha')[[
        'Fecha', 'Temperatura Máxima', 'Temperatura Mínima',
        'Precipitaciones (suma)', 'Dirección del Viento'
    ]]

    df_pivot = pd.merge(df_pivot, clima, on='Fecha', how='left')

    df_pivot = pd.get_dummies(df_pivot, columns=['Dirección del Viento'], prefix='Viento')

    df_pivot = df_pivot.sort_values('Fecha')

    df_pivot['Mes'] = df_pivot['Fecha'].dt.month
    df_pivot['Trimestre'] = df_pivot['Fecha'].dt.quarter

    df_pivot['lag_1'] = df_pivot['Dias Cama Disponibles'].shift(1)
    df_pivot['lag_2'] = df_pivot['Dias Cama Disponibles'].shift(2)
    df_pivot['lag_3'] = df_pivot['Dias Cama Disponibles'].shift(3)

    df_pivot['media_movil_3'] = df_pivot['Dias Cama Disponibles'].rolling(window=3).mean()

    df_pivot['variacion_disponibles'] = df_pivot['Dias Cama Disponibles'].diff()
    df_pivot['porcentaje_ocupacion'] = df_pivot['Dias Cama Ocupados'] / df_pivot['Dias Cama Disponibles']
    df_pivot['ocupados_media_movil'] = df_pivot['Dias Cama Ocupados'].rolling(window=3).mean()
    df_pivot['promedio_media_movil'] = df_pivot['Promedio Cama Disponibles'].rolling(window=3).mean()
    df_pivot['egresos_media_movil'] = df_pivot['Numero de Egresos'].rolling(window=3).mean()

    df_pivot['Diferencia Térmica'] = df_pivot['Temperatura Máxima'] - df_pivot['Temperatura Mínima']

    df_pivot['temp_max_movil'] = df_pivot['Temperatura Máxima'].rolling(window=3).mean()
    df_pivot['precipitacion_movil'] = df_pivot['Precipitaciones (suma)'].rolling(window=3).mean()

    df_pivot['same_month_last_year'] = df_pivot['Dias Cama Disponibles'].shift(12)
    df_pivot['hist_avg_mes'] = df_pivot.groupby('Mes')['Dias Cama Disponibles'].transform('mean')

    df_pivot['interaccion_ocupacion_temp'] = df_pivot['porcentaje_ocupacion'] * df_pivot['Temperatura Máxima']
    df_pivot['interaccion_precipitacion_disp'] = df_pivot['Precipitaciones (suma)'] * df_pivot['Dias Cama Disponibles']

    return df_pivot.dropna().copy()

In [ ]:

def pipeline_final_mae_reducido_arima(hospital):
    print(f"🔍 Extrayendo datos para: {hospital} | Datos Establecimiento")
    # Llamamos a la función de extracción sin el parámetro area, ya que ahora se filtra internamente
    df_base = extraer_info_hospital_area_todo_el_año(hospital)

    if df_base.empty:
        print("❌ No se encontraron datos para el hospital o área especificada.")
        return None

    print("🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...")
    df_modelo = preparar_dataset_climatico_avanzado(df_base)

    if df_modelo is None or df_modelo.empty:
        print("❌ No se pudo preparar el dataset.")
        return None

    # Selección de features
    features = [
        'Dias Cama Ocupados', 'Promedio Cama Disponibles', 'Numero de Egresos',
        'Mes', 'Trimestre', 'lag_1', 'lag_2', 'lag_3', 'media_movil_3',
        'porcentaje_ocupacion', 'variacion_disponibles',
        'ocupados_media_movil', 'promedio_media_movil', 'egresos_media_movil',
        'Temperatura Máxima', 'Temperatura Mínima', 'Precipitaciones (suma)',
        'Diferencia Térmica', 'temp_max_movil', 'precipitacion_movil',
        'same_month_last_year', 'hist_avg_mes',
        'interaccion_ocupacion_temp', 'interaccion_precipitacion_disp'
    ] + [col for col in df_modelo.columns if col.startswith('Viento_')]

    # División temporal
    df_modelo = df_modelo.set_index('Fecha')  # Asegúrate de que la fecha esté como índice
    y = df_modelo['Dias Cama Disponibles']

    # Separar en entrenamiento y test
    train_size = int(len(df_modelo) * 0.8)
    y_train, y_test = y[:train_size], y[train_size:]

    print("🔧 Ajustando modelo ARIMA...")

    # Ajustamos el modelo ARIMA
    model = ARIMA(y_train, order=(5, 1, 0))  # Puedes ajustar el orden (p, d, q)
    model_fit = model.fit()

    print("📊 Evaluando sobre el 20% final de los datos...")

    # Predicción
    y_pred = model_fit.forecast(steps=len(y_test))

    # Cálculo de métricas
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred) ** 0.5
    error_pct = np.mean(np.abs((y_test.values - y_pred) / y_test.values)) * 100

    # Resultado detallado con valores reales y predichos
    resultados = pd.DataFrame({
        'Fecha': y_test.index,
        'Real': y_test.values,                  # Valores reales observados
        'Predicho': y_pred,                     # Valores predichos por el modelo
        'Error Absoluto': np.abs(y_test.values - y_pred),
        'Error Porcentual (%)': np.abs((y_test.values - y_pred) / y_test.values) * 100
    })

    # Reporte resumen
    print(f"✅ MAE (camas): {mae:.2f}")
    print(f"✅ RMSE (camas): {rmse:.2f}")
    print(f"📉 Error promedio porcentual: {error_pct:.2f}%")

    return resultados


In [ ]:
res_ari=pipeline_final_mae_reducido_arima("Hospital Barros Luco Trudeau (Santiago, San Miguel)")
print(res_ari)

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

def pipeline_final_mae_reducido_sarima(hospital):
    print(f"🔍 Extrayendo datos para: {hospital} | Datos Establecimiento")
    # Extracción de datos
    df_base = extraer_info_hospital_area_todo_el_año(hospital)

    if df_base.empty:
        print("❌ No se encontraron datos para el hospital o área especificada.")
        return None

    print("🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...")
    df_modelo = preparar_dataset_climatico_avanzado(df_base)

    if df_modelo is None or df_modelo.empty:
        print("❌ No se pudo preparar el dataset.")
        return None

    features = [
        'Dias Cama Ocupados', 'Promedio Cama Disponibles', 'Numero de Egresos',
        'Mes', 'Trimestre', 'lag_1', 'lag_2', 'lag_3', 'media_movil_3',
        'porcentaje_ocupacion', 'variacion_disponibles',
        'ocupados_media_movil', 'promedio_media_movil', 'egresos_media_movil',
        'Temperatura Máxima', 'Temperatura Mínima', 'Precipitaciones (suma)',
        'Diferencia Térmica', 'temp_max_movil', 'precipitacion_movil',
        'same_month_last_year', 'hist_avg_mes',
        'interaccion_ocupacion_temp', 'interaccion_precipitacion_disp'
    ] + [col for col in df_modelo.columns if col.startswith('Viento_')]

    # División temporal
    df_modelo = df_modelo.set_index('Fecha')
    y = df_modelo['Dias Cama Disponibles']

    train_size = int(len(df_modelo) * 0.8)
    y_train, y_test = y[:train_size], y[train_size:]

    print("🔧 Ajustando modelo SARIMA...")
    # Ajustando el modelo SARIMA
    model = SARIMAX(y_train, order=(5, 1, 0), seasonal_order=(1, 1, 1, 12))  # Ajusta estacionalidad
    model_fit = model.fit()

    print("📊 Evaluando sobre el 20% final de los datos...")

    # Predicción
    y_pred = model_fit.forecast(steps=len(y_test))

    # Cálculo de métricas
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred) ** 0.5
    error_pct = np.mean(np.abs((y_test.values - y_pred) / y_test.values)) * 100

    # Resultado detallado con valores reales y predichos
    resultados = pd.DataFrame({
        'Fecha': y_test.index,
        'Real': y_test.values,
        'Predicho': y_pred,
        'Error Absoluto': np.abs(y_test.values - y_pred),
        'Error Porcentual (%)': np.abs((y_test.values - y_pred) / y_test.values) * 100
    })

    # Reporte resumen
    print(f"✅ MAE (camas): {mae:.2f}")
    print(f"✅ RMSE (camas): {rmse:.2f}")
    print(f"📉 Error promedio porcentual: {error_pct:.2f}%")

    return resultados


In [ ]:
res_sari=pipeline_final_mae_reducido_sarima("Hospital Barros Luco Trudeau (Santiago, San Miguel)")
print(res_sari)

In [ ]:
def extraer_hospitales_validos(filtrado_limpio):
    hospitales = set()
    for df in filtrado_limpio.values():
        if "Nombre Establecimiento" in df.columns and "Area" in df.columns:
            validos = df[df["Area"] == "Datos Establecimiento"][["Nombre Establecimiento"]].dropna().drop_duplicates()
            for _, row in validos.iterrows():
                hospitales.add(row["Nombre Establecimiento"])
    return sorted(list(hospitales))

hospitales_validos = extraer_hospitales_validos(filtrado_limpio)
hospitales_a_evaluar = hospitales_validos[:30]

def pipeline_final_mae_reducido_sarima(hospital):
    print(f"🔍 Extrayendo datos para: {hospital} | Datos Establecimiento")
    df_base = extraer_info_hospital_area_todo_el_año(hospital)

    if df_base.empty:
        print("❌ No se encontraron datos para el hospital o área especificada.")
        return None

    print("🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...")
    df_modelo = preparar_dataset_climatico_avanzado(df_base)

    if df_modelo is None or df_modelo.empty:
        print("❌ No se pudo preparar el dataset.")
        return None

    features = [
        'Dias Cama Ocupados', 'Promedio Cama Disponibles', 'Numero de Egresos',
        'Mes', 'Trimestre', 'lag_1', 'lag_2', 'lag_3', 'media_movil_3',
        'porcentaje_ocupacion', 'variacion_disponibles',
        'ocupados_media_movil', 'promedio_media_movil', 'egresos_media_movil',
        'Temperatura Máxima', 'Temperatura Mínima', 'Precipitaciones (suma)',
        'Diferencia Térmica', 'temp_max_movil', 'precipitacion_movil',
        'same_month_last_year', 'hist_avg_mes',
        'interaccion_ocupacion_temp', 'interaccion_precipitacion_disp'
    ] + [col for col in df_modelo.columns if col.startswith('Viento_')]

    df_modelo = df_modelo.set_index('Fecha')
    y = df_modelo['Dias Cama Disponibles']

    train_size = int(len(df_modelo) * 0.8)
    y_train, y_test = y[:train_size], y[train_size:]

    print("🔧 Ajustando modelo SARIMA...")
    model = SARIMAX(y_train, order=(5, 1, 0), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit()

    print("📊 Evaluando sobre el 20% final de los datos...")

    y_pred = model_fit.forecast(steps=len(y_test))

    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred) ** 0.5
    error_pct = np.mean(np.abs((y_test.values - y_pred) / y_test.values)) * 100

    resultados = pd.DataFrame({
        'Fecha': y_test.index,
        'Real': y_test.values,
        'Predicho': y_pred,
        'Error Absoluto': np.abs(y_test.values - y_pred),
        'Error Porcentual (%)': np.abs((y_test.values - y_pred) / y_test.values) * 100
    })

    print(f"✅ MAE (camas): {mae:.2f}")
    print(f"✅ RMSE (camas): {rmse:.2f}")
    print(f"📉 Error promedio porcentual: {error_pct:.2f}%")

    return resultados

resultados_todos = {}

for hospital in hospitales_a_evaluar:
    try:
        print("\n==========================")
        print(f"🏥 Evaluando: {hospital}")
        print("==========================")
        resultados = pipeline_final_mae_reducido_sarima(hospital)
        if resultados is not None:
            resultados_todos[hospital] = resultados
    except Exception as e:
        print(f"❌ Error en {hospital}: {e}")
        continue

tabla_resumen = []

for hospital, resultados in resultados_todos.items():
    if "Error Absoluto" in resultados.columns and "Error Porcentual (%)" in resultados.columns:
        mae = resultados["Error Absoluto"].mean()
        error_pct = resultados["Error Porcentual (%)"].mean()
        tabla_resumen.append({
            "Hospital": hospital,
            "MAE (camas)": round(mae, 2),
            "Error Porcentual Promedio (%)": round(error_pct, 2)
        })

df_tabla_resumen = pd.DataFrame(tabla_resumen).sort_values("MAE (camas)").reset_index(drop=True)
display(df_tabla_resumen)


In [ ]:

import lightgbm as lgb

def pipeline_final_mae_reducido_lightgbm(hospital):
    print(f"🔍 Extrayendo datos para: {hospital} | Datos Establecimiento")
    df_base = extraer_info_hospital_area_todo_el_año(hospital)

    if df_base.empty:
        print("❌ No se encontraron datos para el hospital o área especificada.")
        return None

    print("🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...")
    df_modelo = preparar_dataset_climatico_avanzado(df_base)

    if df_modelo is None or df_modelo.empty:
        print("❌ No se pudo preparar el dataset.")
        return None

    features = [
        'Dias Cama Ocupados', 'Promedio Cama Disponibles', 'Numero de Egresos',
        'Mes', 'Trimestre', 'lag_1', 'lag_2', 'lag_3', 'media_movil_3',
        'porcentaje_ocupacion', 'variacion_disponibles',
        'ocupados_media_movil', 'promedio_media_movil', 'egresos_media_movil',
        'Temperatura Máxima', 'Temperatura Mínima', 'Precipitaciones (suma)',
        'Diferencia Térmica', 'temp_max_movil', 'precipitacion_movil',
        'same_month_last_year', 'hist_avg_mes',
        'interaccion_ocupacion_temp', 'interaccion_precipitacion_disp'
    ] + [col for col in df_modelo.columns if col.startswith('Viento_')]

    df_modelo = df_modelo.set_index('Fecha')
    y = df_modelo['Dias Cama Disponibles']
    X = df_modelo[features]

    # División de los datos en entrenamiento y prueba
    train_size = int(len(df_modelo) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    print("🔧 Ajustando modelo LightGBM...")

    # Ajustando el modelo LightGBM
    model = lgb.LGBMRegressor(objective='regression', metric='l2', boosting_type='gbdt')
    model.fit(X_train, y_train)

    print("📊 Evaluando sobre el 20% final de los datos...")

    # Predicción
    y_pred = model.predict(X_test)

    # Cálculo de métricas
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred) ** 0.5
    error_pct = np.mean(np.abs((y_test.values - y_pred) / y_test.values)) * 100

    # Resultado detallado con valores reales y predichos
    resultados = pd.DataFrame({
        'Fecha': y_test.index,
        'Real': y_test.values,
        'Predicho': y_pred,
        'Error Absoluto': np.abs(y_test.values - y_pred),
        'Error Porcentual (%)': np.abs((y_test.values - y_pred) / y_test.values) * 100
    })

    # Reporte resumen
    print(f"✅ MAE (camas): {mae:.2f}")
    print(f"✅ RMSE (camas): {rmse:.2f}")
    print(f"📉 Error promedio porcentual: {error_pct:.2f}%")

    return resultados

In [75]:
res_lgbm = pipeline_final_mae_reducido_lightgbm("Hospital Barros Luco Trudeau (Santiago, San Miguel)")
print(res_lgbm)


🔍 Extrayendo datos para: Hospital Barros Luco Trudeau (Santiago, San Miguel) | Datos Establecimiento
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo LightGBM...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001033 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 738
[LightGBM] [Info] Number of data points in the train set: 97, number of used features: 26
[LightGBM] [Info] Start training from score 22636.958763
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Light